# Using a SurrealDB Knowedge Graph and Gemini to build a Graph-RAG 

This Jupyter Notebook illustrates the implementation and utilization of a graph-based Retrieval-Augmented Generation (RAG) system using a knowledge graph stored in SurrealDB. It focuses on leveraging the knowledge graph created by the graph_extractor.ipynb notebook for question answering tasks.

### Key Functionality
####  Data Loading:
        Loads the raw text data from which will be used for context in the prompt.

#### Leveraging the Existing Knowledge Graph:

        Create a SurrQL function to query the knowledge graph using vector search for the descriptions of the entities and the other entities and relationships they have in the graph. This data will also be fed into the prompt for context.

        
#### Response Generation without a knowledge graph:
    Using a template prompt in the RAG_prompts.py file a simple context window of the text data loaded earlier with the question is asked of the LLM model.

    

#### Response Generation with knowledge graph:
    Using a template prompt in the RAG_prompts.py file a context window of the text data loaded earlier, plus the graph data retrieved from SurrealDB with the question is asked of the LLM model.


##### notes:
    This notebook utilizes libraries :
        surrealdb to interact with SurrealDB
        google.generativeai to interact with Gemini

    Prerquisite is to install the embedding model as in this python script:
            https://github.com/apireno/surrealDB_embedding_model
            and load a knowledge graph using the other notebook in this repo: graph_extractor.ipynb

    The notebook is loosely based on a larger repository microsoft/graphrag. The microsoft/graphrag repository provides a more comprehensive, modular graph-based RAG system. This notebook focuses specifically on the knowledge graph construction aspect.

In [1]:
import sys
import os
import json
import google.generativeai as genai
import os
import time
import ipynb_path
import pandas as pd
from surrealdb import AsyncSurreal

from IPython.display import display, Markdown,JSON
from IPython.display import JSON


#get this notebook's path for access to the other files needed
dir_path = os.path.dirname(os.path.realpath(ipynb_path.get(__name__)))
sys.path.append(dir_path) #add the current directory for adding py imports
from RAG_prompts import STD_RAG_PROMPT, GRAPH_RAG_PROMPT

In [2]:

# this folder
nb_folder = dir_path
input_file = nb_folder + "/Operation Dulce v2 1 1.md"

out_folder = nb_folder + "/rag_{0}".format(time.strftime("%Y%m%d-%H%M%S"))
os.makedirs(out_folder, exist_ok=True)
chat_file = out_folder + "/chat.json"

messages_to_log = []



ip = "0.0.0.0:8000"
url = "ws://{0}".format(ip)

u = "root"
p = "root"
n = "graph_rag"
d = "graph_rag"
db_folder = nb_folder + "/db"

surrealdb_start = "surreal start --allow-net --log none --user {u} --pass {p} --bind {ip} \"rocksdb://{db_folder}\"".format(
    u=u,
    p=p,
    ip=ip,
    db_folder=db_folder)

#run this command if your surreal instance isn't running yet 
#copy and paste from below into a terminal
print(surrealdb_start)

#and ensure you installed the embedding model!
#the model will power the function fn::sentence_to_vector($text)
print("""
ensure you installed the embedding model!
      
      https://github.com/apireno/surrealDB_embedding_model

the model will power the function fn::sentence_to_vector($text)
"""   )  

surreal start --allow-net --log none --user root --pass root --bind 0.0.0.0:8000 "rocksdb:///Users/sandro/git_repos/graph_rag/db"

ensure you installed the embedding model!
      
      https://github.com/apireno/surrealDB_embedding_model

the model will power the function fn::sentence_to_vector($text)



In [24]:

# get the text from the source file for use in our prompt
with open(input_file, "r") as f:
    input_text = f.read()


entity_types = ["PERSON", "PLACE"]
relation_type = "RELATED_TO"

#create the function that will return the portion of the knowledge graph 
#relevant to the question using vector search on the description embedding
retrieve_function_surql = """
DEFINE FUNCTION OVERWRITE fn::get_related_entities($text: string) {{
    
    LET $emb = fn::sentence_to_vector($text);
    
    RETURN SELECT id,description, ->{1}.{{out,strength,description}} FROM {0} 
        WHERE  embedding <|5,COSINE|> $emb;
}};
""".format(entity_types[0],relation_type)


async with AsyncSurreal(url) as db:
    await db.signin( {"username":u, "password":p}) 
    await db.use(n, d)
    outcome = await db.query(retrieve_function_surql)



In [3]:

#This is a helper function to make the knowledge graph a bit easier to read.
def transform_record_ids(data):
  """
  Transforms the input data to extract the 'ID' field from 'out' and 'in' objects.

  Args:
    data: An array of dictionaries.

  Returns:
    The transformed array of dictionaries.
  """
    
  for item in data:
    if "id" in item:
        item["id"] = item["id"].id
    if "->RELATED_TO" in item:
      for inner_dict in item["->RELATED_TO"]:
        if "out" in inner_dict and hasattr(inner_dict["out"], "id"):
          inner_dict["out"] = inner_dict["out"].id
        if "in" in inner_dict and hasattr(inner_dict["in"], "id"):
          inner_dict["in"] = inner_dict["in"].id
        if "id" in inner_dict and hasattr(inner_dict["id"], "id"):
          inner_dict["id"] = inner_dict["id"].id
      
    
  return data


In [7]:

#Let's test out vector/graph query
#we should see a collection of people and relationships
#where the person's description is related to the question 

async with AsyncSurreal(url) as db:
    await db.signin( {"username":u, "password":p}) 
    await db.use(n, d)
    get_the_graph_context_surql = "fn::get_related_entities('{0}')"
    outcome = await db.query(get_the_graph_context_surql.format("Who is the main character?"))

result = transform_record_ids(outcome)
#print(result[0])

#df["out"] = df["out"].apply(lambda x: x.id) 
#print(df["->RELATED_TO"].apply(lambda x: pd.DataFrame(x)["out"]))
#JSON(result)
print(json.dumps(result, indent=4, sort_keys=True))
    

[
    {
        "->RELATED_TO": [
            {
                "description": "Alex Mercer operates within Dulce Base during Operation: Dulce.",
                "out": "DULCE BASE",
                "strength": 9
            },
            {
                "description": "Alex Mercer and Taylor Cruz are both members of the Paranormal Military Squad and work together on Operation: Dulce.",
                "out": "TAYLOR CRUZ",
                "strength": 8
            },
            {
                "description": "Alex Mercer reports to and receives orders from Washington D.C.",
                "out": "WASHINGTON",
                "strength": 6
            },
            {
                "description": "Alex Mercer and Jordan Hayes are both members of the Paranormal Military Squad and work closely together, sharing a mutual respect and understanding.",
                "out": "JORDAN HAYES",
                "strength": 9
            },
            {
                "description": "Al

In [9]:
#these functions will call gemini api based on our prompt templates and input


#input object of the model, reference to the attached file and the prompt content
def generate_content_with_attachment(the_model,the_messages,attached_file):
    #time.sleep(5) # to stop generating too many requests per min

    response = the_model.generate_content([attached_file,str(the_messages)])
    return response





#this generates the prompt for a std RAG approach where we are supplying the document with the question
def generate_std_LLM_messages(the_question,attached_file):
    messages =  [
            {"role": "user", "parts": [{"text": STD_RAG_PROMPT.format(
                question = the_question
            )}]}
        ]
    return messages


#this function will call the LLM with a std RAG approach prompt 
def get_std_LLM_response(the_model,the_question,attached_file):
    
    messages = generate_std_LLM_messages(the_question,attached_file)
    messages_to_log.append(messages)
    
    with open(chat_file, "w") as f:
        json.dump(messages_to_log, f)
        
    response = generate_content_with_attachment(the_model,messages,attached_file)
    return response


#this generates the prompt for a graphRAG approach where we are supplying the document and knowledge graph with the question
#it calls the SurQL function get_related_entities to do a vector search based on the input question   
async def generate_graph_RAG_messages(the_db,the_question,attached_file):
    get_the_graph_context_surql = "fn::get_related_entities('{0}')"
    
    outcome = await the_db.query(get_the_graph_context_surql.format(the_question))
    messages = [
            {"role": "user", "parts": [{"text": GRAPH_RAG_PROMPT.format(
                question = the_question,
                knowledge_graph = outcome
            )}]}
        ]
    
    return messages
    
 
#this function will call the LLM with a prompt that includes the text of the original document
#plus the relevant entities and relationships from the knowledge graph based on a vector search
async def get_graph_RAG_response(the_db,the_model,the_question,attached_file):
    
    messages = await generate_graph_RAG_messages(the_db,the_question,attached_file)
    messages_to_log.append(messages)
    
    with open(chat_file, "w") as f:
        json.dump(messages_to_log, f)
    
    response = generate_content_with_attachment(the_model,messages,attached_file)
    return response
    
       

In [10]:

# Set your API key (I've stored mine in an env variable)
genai.configure(api_key=os.getenv('GOOGLE_GENAI_API_KEY'))

#flash is faster and good enough
#model = "gemini-1.0-pro"
model = "gemini-1.5-flash"

model = genai.GenerativeModel(model)


the_attached_file = genai.upload_file(input_file)

while the_attached_file.state.name == "PROCESSING":
    print(".", end="")
    time.sleep(10)
    the_attached_file = genai.get_file(the_attached_file.name)





In [11]:
#Let's see how the prompts will look
#You can copy and paste these into any LLM chat engine to test if you also supply the document text


q = "Who is the main character and what motivates them?"

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")
messages = generate_std_LLM_messages(q,the_attached_file)
print(json.dumps(messages[0], indent=4, sort_keys=True))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with AsyncSurreal(url) as db:
    await db.signin( {"username":u, "password":p}) 
    await db.use(n, d)
    messages = await generate_graph_RAG_messages(db,q,the_attached_file)
    print(json.dumps(messages[0], indent=4, sort_keys=True))
 
   






 ------ STD RAG -------------------------




{
    "parts": [
        {
            "text": "\n-Goal-\nYou are a researcher who is providing answers to a question for you client based on documents provided you. Answer the questions to the best of your ability given the documents.\n\n<question>\nWho is the main character and what motivates them?\n</question>\n\n "
        }
    ],
    "role": "user"
}




 ------ GRAPH RAG -------------------------




{
    "parts": [
        {
            "text": "\n-Goal-\nYou are a researcher who is providing answers to a question for your client based on documents provided you. You also have access to a knowledge graph based on the documents provided. Answer the questions to the best of your ability given the documents and knowledge graph.\n\n<knowledge graph>\n[{'->RELATED_TO': [{'description': 'Alex Mercer operates within Dulce Base during Operation: Dulce.', 'out': RecordID(table_name=PLACE, record_id=DULCE BASE), 'strength': 9}, {'descript

In [13]:
#Lets test a question on doc alone ...

q = "Who is the main character and what motivates them? Format your response in bullet points."

response = get_std_LLM_response(model,q,the_attached_file)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")

display(Markdown(response.text))









 ------ STD RAG -------------------------






* **Main Character:** While the story features a team, Agent Alex Mercer arguably functions as the main character.  He's the focal point of several scenes and his internal conflicts and reflections are prominently featured.

* **Motivations:**
    * **Duty and Obedience:** Mercer is initially driven by a sense of duty and obedience to his superiors, exemplified by his initial compliance with Agent Cruz's commands.  This is a learned response rooted in his military background.

    * **Uncovering the Truth:**  A deeper motivation, however, emerges as the narrative progresses.  Mercer demonstrates a strong desire to understand the truth behind Operation Dulce, going beyond simple adherence to protocol. This is evident in his quiet observations of Jordan and Sam, and his willingness to question established narratives.

    * **Protecting Humanity:** Mercer's actions suggest a desire to protect humanity from any potential threat emanating from Dulce, even if this means defying or challenging authority.

    * **Collaboration and Understanding:**  As the operation unfolds, Mercer's motivation shifts towards collaboration and understanding. He actively seeks the insights of Jordan and Sam, recognizing their expertise is crucial to the team’s success.  His leadership style evolves from command-and-control to one of collaborative engagement.  He advocates for a cautious, diplomatic approach to interacting with the extraterrestrial intelligence.


In [14]:
# and with the graph?
print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with AsyncSurreal(url) as db:
    await db.signin( {"username":u, "password":p})  
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,the_attached_file)
    display(Markdown(response.text))





 ------ GRAPH RAG -------------------------






* **Main Character:** Agent Alex Mercer is the main character, although the narrative provides perspectives from other team members.

* **Motivations:** Alex Mercer is motivated by several factors:

    * **Duty and Responsibility:**  He feels a strong sense of duty to complete his mission and protect his team. His military background emphasizes following protocol, but he's also shown to be willing to deviate when necessary.  His leadership consistently emphasizes the safety and success of the mission and his team members.
    * **Curiosity and a thirst for knowledge:** He is intensely curious about the mysteries surrounding Operation: Dulce and seeks to understand them, even questioning established narratives.  He's willing to consider speculative theories when the situation demands it.
    * **Teamwork and Camaraderie:**  He values his team and works to foster collaboration and trust, even among those with conflicting personalities.  He develops a close working relationship and mutual respect with Jordan Hayes. He also acts as a mentor for Sam Rivera.  He seeks to leverage his team's diverse expertise to succeed.
    * **Uncovering Truth:** There's an underlying desire to understand the true nature of Operation: Dulce.  He isn't just following orders; he's actively seeking truth and knowledge.  He shows a willingness to challenge authority when necessary to pursue this goal.



In [16]:

#these helper functions will run both approaches on the same question and print them for us

    
async def process_and_display_RAG_comparison(model,q,the_attached_file):
  
    response = get_std_LLM_response(model,q,the_attached_file)
    
    print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")
    
    display(Markdown(response.text))

    
    print ("""------ GRAPH RAG -------------------------
""")
        
    async with AsyncSurreal(url) as db:
        await db.signin( {"username":u, "password":p})  
        await db.use(n, d)
        response = await get_graph_RAG_response(db,model,q,the_attached_file)
        display(Markdown(response.text))

    

In [18]:

#Let's test a question that asks about the main character's relationships
q = "What places and people does Alex Mercer hold dearest? Format your response in bullet points. Format your response in bullet points."
await process_and_display_RAG_comparison(model,q,the_attached_file)






 ------ STD RAG -------------------------






Based on the provided text, Alex Mercer's dearest places and people are not explicitly stated.  The narrative focuses on his professional life and his role within the team during Operation: Dulce.  While he shows camaraderie and respect for his team members (Jordan Hayes, Taylor Cruz, and Sam Rivera),  the text does not offer insight into his personal life or relationships outside of his work.  Therefore, a list of places and people he holds dearest cannot be created from this document.


------ GRAPH RAG -------------------------



Based on the provided text and knowledge graph, here's what we can infer about the places and people Alex Mercer holds dearest:

* **Dulce Base:** The knowledge graph shows a strong connection (strength 9) between Alex Mercer and Dulce Base, indicating his significant involvement and time spent there during Operation: Dulce.  The narrative strongly suggests a sense of shared mission and experience with his team within the base.

* **Jordan Hayes:** The knowledge graph highlights a very strong relationship (strength 9) between Alex and Jordan.  The text emphasizes their mutual respect, shared understanding, and close collaboration.  This suggests a deep professional and possibly personal bond.

* **Washington D.C.:** While not as strongly connected as Dulce Base or Jordan Hayes (strength 6), Washington D.C. represents Alex's chain of command and the source of his mission. His loyalty and adherence to protocols suggest a sense of duty towards this authority.

* **Sam Rivera:**  The knowledge graph shows a strong connection (strength 7) between Alex and Sam. The text reveals that Alex mentors Sam, suggesting care and investment in Sam's development.  This suggests a mentor-mentee relationship that likely involves a degree of respect and concern.

* **Taylor Cruz:** The knowledge graph shows a strong connection (strength 8) between Alex and Taylor.  The text suggests that their relationship is complex, with Taylor's authoritative style sometimes clashing with Alex's independent thinking.  While not explicitly stated as "dearest," their close working relationship in a high-stakes situation suggests a certain level of professional respect and interdependence.


It's important to note that "dearest" is subjective. This analysis focuses on the strength of relationships and emotional connections implied in the text and knowledge graph.  The narrative emphasizes Alex's professional dedication, and so his "dearest" relationships are likely strongly tied to his work and those vital to mission success.


In [19]:


q = """


Create a table of places that Alex communicates with:


In the first column indicate the person. 
In the 2nd column list the frequency he has scenes with them.
In the 3rd column list the places they had scenes together ordered by importance of the place and scene.
In the 4th column indicate the strength of the bond between the people.
In the 5th column score the total weight of the interactions across the scenes in aggregate.

output the response in a table format using markdown language.
"""

await process_and_display_RAG_comparison(model,q,the_attached_file)






 ------ STD RAG -------------------------






## Alex Mercer's Communication Network

| Person             | Communication Frequency | Places of Interaction (Ordered by Importance)                                                                     | Bond Strength | Interaction Weight Score |
|----------------------|--------------------------|-----------------------------------------------------------------------------------------------------------------|-----------------|------------------------|
| Jordan Hayes         | Very High                 | Dulce Base Command Center (multiple scenes, crucial discussions, collaborative work), Dulce Base Corridors (shared concerns), Elevator to Dulce (shared apprehension), Deserted Corridor (brief exchange), Briefing Room (initial meeting) | Very Strong      | 9                        |
| Taylor Cruz          | High                      | Dulce Base Command Center (multiple scenes, contrasting views), Dulce Base Corridors (conflicts, brief agreements), Briefing Room (initial instructions), Equipment Bay (brief tension), Elevator to Dulce (brief exchange) | Strong but Tense   | 7                        |
| Sam Rivera           | High                      | Dulce Base Command Center (multiple scenes, collaborative work), Sam's Office (advice, encouragement), Briefing Room (initial briefing) | Strong             | 6                        |


**Note:** The "Interaction Weight Score" is a subjective assessment based on the length and significance of interactions described in the provided text.  A higher score indicates more frequent and/or crucial interactions.  The ordering of places within the "Places of Interaction" column is also based on a subjective interpretation of the narrative's emphasis.


------ GRAPH RAG -------------------------



Based on the provided text, here's a table summarizing Alex Mercer's communications,  keeping in mind that "scenes" are inferred from the narrative and the strength of the bond is a subjective assessment based on the text's portrayal of their relationships.  The "total weight" is a combined score—a higher number indicating more significant and frequent interactions.  It's not a precise calculation but a qualitative assessment.

| Person             | Frequency of Scenes | Places (Ordered by Importance)                                  | Bond Strength | Total Weight |
|----------------------|----------------------|--------------------------------------------------------------------|-----------------|---------------|
| Jordan Hayes         | Very High            | Dulce Base (Command Center, Corridors, Labs), Elevator, Briefing Room | 9               | 10            |
| Sam Rivera           | High                 | Dulce Base (Command Center, Server Room), Briefing Room, Office     | 7               | 8             |
| Taylor Cruz          | High                 | Dulce Base (Command Center, Corridors, Equipment Bay), Briefing Room | 8               | 9             |
| Washington D.C.     | Moderate             | Briefing Room                                                    | 6               | 6             |


**Note:**  The frequency and importance of places are subjective interpretations based on the narrative. The text doesn't offer precise counts of interactions in each location.  The bond strength is an estimate of the closeness and mutual respect shown between characters, based on my interpretation of the text. The "Total Weight" is a subjective assessment combining the frequency, importance of places, and bond strength to give an overall sense of the interaction's significance.


In [20]:


q = """

Create a table of places that Alex visits:


In the first column indicate the place name. 
In the 2nd column indicate the frequency he visits them.
In the 3rd column list the people who were there with him by strength of the bond of the people.
In the 4th column indicate the importance of the place.
In the 5th column describe the location in 255 characters or less.
In the 6th column score the total weight of the interactions across the scenes in aggregate.

output the response in a table format using markdown language.
"""


await process_and_display_RAG_comparison(model,q,the_attached_file)






 ------ STD RAG -------------------------






## Alex Mercer's Locations in Operation: Dulce

| Place Name             | Visit Frequency | People Present (Bond Strength)                  | Importance                               | Location Description                                                                     | Interaction Weight Score |
|-------------------------|-----------------|-------------------------------------------------|-------------------------------------------|-----------------------------------------------------------------------------------------|--------------------------|
| Briefing Room           | 2+              | Taylor Cruz (Antagonistic), Jordan Hayes (Allied), Sam Rivera (Allied) | High (Mission planning, strategy)     | Sterile, high-tech room for mission briefings.                                          | 8                         |
| Dulce Base - Corridors | Multiple         | Jordan Hayes (Allied), Taylor Cruz (Antagonistic), Sam Rivera (Allied) | High (Main setting of operation)       | Long, dark, and ominous corridors of the underground base.                               | 10                        |
| Dulce Base - Lab       | 1                | Jordan Hayes (Allied)                          | Medium (Alien tech analysis)           | Sterile laboratory for examining alien artifacts.                                        | 5                         |
| Dulce Base - Server Room| 1                | Sam Rivera (Allied)                           | Medium (Data analysis)                  | Cramped room filled with servers and computing equipment.                                | 4                         |
| Dulce Base - Elevator Shaft | 1                | Jordan Hayes (Allied), Taylor Cruz (Antagonistic), Sam Rivera (Allied) | High (Transportation to Dulce)         | Metal shaft leading into Dulce base. The elevator felt a disturbance within the walls    | 6                         |
| Dulce Base - Equipment Bay | 1                | Taylor Cruz (Antagonistic), Jordan Hayes (Allied) | Low (Gear check)                         | Secluded area where Cruz checked mission equipment.                                   | 3                         |
| Dulce Base - Mainframe Room | 2+              | Jordan Hayes (Allied), Taylor Cruz (Antagonistic), Sam Rivera (Allied)  | High (Central hub, communication)  | Imposing doors to the mainframe room, possibly an invitation or trap.                | 10                        |
| Dulce Base - Communications Hub | 2+           | Jordan Hayes (Allied), Taylor Cruz (Antagonistic), Sam Rivera (Allied) | High (Signal analysis and communication) | Main communications hub; contains central servers and consoles for data processing.     | 10                         |
| Dulce Base - Command Center | Multiple | Jordan Hayes (Allied), Taylor Cruz (Antagonistic), Sam Rivera (Allied) | High (Mission control, final communication) | Central control room of Dulce Base; advanced consoles and monitors.                  | 12                        |


**Bond Strength Key:**

* **Allied:** Strong mutual trust and collaboration.
* **Antagonistic:**  Conflict or tension; differing goals or approaches.


**Interaction Weight Score:** This is a subjective assessment based on the length and importance of scenes in which Alex interacts with others at each location.  Higher scores indicate more significant interactions.


------ GRAPH RAG -------------------------



| Place Name        | Frequency | People (by strength)                     | Importance          | Location Description                                                                                                | Total Weight |
|--------------------|-----------|------------------------------------------|----------------------|-------------------------------------------------------------------------------------------------------------------|-------------|
| Dulce Base         | High       | Jordan Hayes (9), Taylor Cruz (8), Sam Rivera (7) | Extremely High      | Subterranean military base; mysterious, technologically advanced, holds many secrets.                               | 36          |
| Washington         | Low        | N/A                                      | High                 | US Capital; headquarters for Paranormal Military Squad; provides orders and support for Operation: Dulce           | 12          |
| Briefing Room (Dulce)| High       | Jordan Hayes, Taylor Cruz, Sam Rivera    | High                 | Sterile briefing room inside Dulce Base, where mission briefings take place.                                      | 24          |
| Briefing Room (Washington)| Low       | N/A                                      | Medium               | Sterile briefing room in Washington D.C. where operational briefings take place.                              | 6           |
| Cramped Office      | Low        | Sam Rivera                             | Medium               | Small office in the facility, likely near Dulce Base, contains multiple servers for Sam Rivera's data analysis. | 7           |
| Deserted Corridor   | Low        | Taylor Cruz                             | Low                  | Long, empty corridor in the Dulce Base.                                                                          | 9          |
| Secluded Equipment Bay| Low        | Taylor Cruz, Jordan Hayes                  | Low                  | Room in the facility where equipment is stored.                                                                   | 10          |
| Cramped Underground Meeting Room | Low     | Jordan Hayes, Taylor Cruz, Sam Rivera    | High                | Small room, underground, used by team to strategize before entering Dulce Base.                                     | 24          |
| Elevator Shaft      | Low        | Jordan Hayes, Taylor Cruz, Sam Rivera    | Medium               | Artificial shaft leading down to Dulce Base.                                                                 | 24          |
| Central Communications Hub | Low | Jordan Hayes, Alex Mercer, Taylor Cruz | High | Main communications center deep within Dulce Base, where alien communication is received.            | 24          |
| Mainframe Room      | Low        | Jordan Hayes, Alex Mercer, Taylor Cruz, Sam Rivera | High | Central server room of Dulce Base, containing the mainframe.                                                 | 34          |
| Lab                  | Low        | Jordan Hayes, Alex Mercer                 | High                 | Laboratory within Dulce Base where the team analyzes alien artifacts.                                              | 19          |
| Server Room          | Low        | Alex Mercer, Sam Rivera                    | High                 | Room with numerous servers, used for data analysis related to Operation: Dulce.                                | 17          |
| Central Communications Hub (Surface) | Low | Alex Mercer, Jordan Hayes, Taylor Cruz | High | Surface facility adjacent to Dulce Base, acts as a communications relay point for the mission. | 24          |


**Note:**  The "Total Weight" is a *rough estimate* based on the strength of relationships mentioned in the story and the frequency of locations mentioned.  It's not a precise metric.  Also, some locations are implied rather than explicitly stated; their weight and frequency are therefore subjective interpretations of the narrative.


In [21]:



q = """
    Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Describe this cover in 20 words or less.
"""
await process_and_display_RAG_comparison(model,q,the_attached_file)







 ------ STD RAG -------------------------






Dulce Base command center: Four figures bathed in alien light, deciphering a cosmic message.


------ GRAPH RAG -------------------------



Here's a description for the book cover, fulfilling the prompt's requirements:

**Image:** Alex Mercer, Jordan Hayes, silhouetted against Dulce Base's glowing entrance, a cosmic signal pulsing behind them.

**Words:**  Dulce Base:  First contact.  A battle for humanity's future.


In [22]:


q = """Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Draw this cover in the syle of an 8-bit video game.
"""
await process_and_display_RAG_comparison(model,q,the_attached_file)






 ------ STD RAG -------------------------






The most emotionally charged and important scene for the book cover, considering Alex Mercer's strongest relationships and key plot locations, would be the moment in the Dulce Base command center when the team first establishes a reciprocal, responsive communication with the alien intelligence.  This scene encapsulates several key elements:

* **Strongest Relationships:** The cover would show Alex Mercer, Jordan Hayes, Taylor Cruz, and Sam Rivera together, highlighting their collaborative effort and the bonds forged under pressure. The scene emphasizes their unity amidst the unknown.

* **Key Plot Location:** The Dulce Base command center is a central location throughout the story, symbolizing the heart of the mystery and the site of their crucial breakthrough.

* **Emotional Charge:** This moment is the culmination of their efforts, the point where they move beyond passive observation to active communication with an extraterrestrial intelligence. The emotional charge comes from the uncertainty, the risk, and the sheer enormity of the achievement. It is the point of no return, where their actions will irrevocably shape the future.


**8-bit Video Game Cover Art Description:**

The cover art would be rendered in a classic 8-bit style, reminiscent of early NES or SNES games.

* **Background:** A stylized, pixelated representation of the Dulce Base command center.  The walls would be dark grey/blue blocks, with various shades of grey and blue used to suggest depth and texture.  The consoles would be brightly colored rectangles with pixelated lights flashing on and off in different colors (greens, blues, reds).  There could be a subtle pixelated starfield visible through a partially transparent window, suggesting the extraterrestrial connection.

* **Characters:** Alex Mercer would be positioned slightly forward, central, depicted as a slightly taller, more detailed sprite than the others, showing his leadership role. He could be holding a handheld device or a console controller, suggesting active participation in communication. Jordan Hayes, Taylor Cruz, and Sam Rivera would be positioned slightly behind Alex, with slightly smaller sprites, suggesting a team working in unison.  Each would have unique color palettes (Jordan could be teal, Taylor maybe purple, Sam perhaps bright orange) to distinguish them easily.  They would each be interacting with their respective consoles or displays.

* **Alien Signal:** The alien signal could be represented as a stream of vividly colored pixels flowing from a console or screen toward the characters, creating a visual bridge between them and the extraterrestrial intelligence. The colors should be vibrant and slightly otherworldly (think purples, pinks, and electric blues).

* **Title:**  "Operation: Dulce" would be displayed at the top in a bold, pixelated 8-bit font, possibly with a subtitle beneath like "First Contact."

* **Overall Style:** The palette would be a blend of dark and bright colors, creating a sense of both tension and excitement.  The pixel art style would be crisp and clean, with a focus on clear silhouettes and contrasting colors to make the characters and the key elements of the scene stand out.  The overall mood would be one of anticipation, mystery, and slight apprehension, reflecting the essence of the story.


------ GRAPH RAG -------------------------



Here's a description of an 8-bit video game style book cover, based on the provided text and knowledge graph:


**Book Cover Title:** Operation: Dulce


**Style:** 8-bit pixel art, reminiscent of classic NES or SNES RPGs.


**Scene Depicted:** The cover art depicts the climax of the novel, where the team is in the Dulce Base command center engaging in their first interstellar communication.


**Elements:**

* **Background:** A pixelated depiction of the Dulce Base command center.  Use a dark blue/grey color palette for the walls and floors. Monitors displaying streams of alien code should dominate the background, using various shades of green and blue pixels to create a sense of technology and mystery.  A few flickering pixels suggest the power surges described.

* **Characters:**

    * **Alex Mercer:**  In the center foreground, slightly larger than the other characters to denote his leadership.  Depict him in pixelated 8-bit military attire (dark green jacket, darker pants) looking intently at a monitor, holding his rifle loosely, suggesting a balance of readiness and focused engagement.  Give him a determined expression with fairly detailed pixel eyes.

    * **Jordan Hayes:** To Alex's left, slightly smaller but still prominent. They are dressed in more scientific attire (a white lab coat).  Their head is tilted toward a monitor, highlighting their role in deciphering the alien code, and their expression shows intense concentration.

    * **Taylor Cruz:**  To Alex's right, slightly smaller and similarly outfitted as Alex but with more angular features to reflect their more severe personality. Their expression shows a blend of apprehension and calculated determination.

    * **Sam Rivera:**  Smaller than the others, positioned slightly behind and between Alex and Taylor, hunched over a computer console displaying alien symbols. Their 8-bit design could include bright, colorful clothes to represent their youthful energy and tech-savvy nature. Their expression could be one of excited concentration.

* **Color Palette:** Primarily dark blues, greens, and greys for the background and characters' attire, representing the shadowy base. Brighter greens, blues, and possibly yellows should be used for the monitors to highlight the alien code.


**Overall Impression:** The cover should evoke a sense of mystery, tension, and impending discovery. The pixel art style should be clean and well-defined, avoiding too much detail to maintain the 8-bit aesthetic. The limited color palette should enhance the mood and style.  The overall scene should focus on the team in the midst of interaction with alien intelligence, highlighting both the excitement and the danger. The title "Operation: Dulce" should be prominently displayed in a classic 8-bit font, possibly with a pixelated glow effect.


In [80]:

#unfortunately we don't have access to imagen yet unless you are on the beta testing team.. leaving this here for future
imagen = genai.ImageGenerationModel("imagen-3.0-generate-001")

result = imagen.generate_images(
    prompt="Fuzzy bunnies in my kitchen",
    number_of_images=4,
    safety_filter_level="block_only_high",
    person_generation="allow_adult",
    aspect_ratio="3:4",
    negative_prompt="Outside",
)

for image in result.images:
  print(image)

# Open and display the image using your local operating system.
for image in result.images:
  image._pil_image.show()





NotFound: 404 models/imagen-3.0-generate-001 is not found for API version v1beta, or is not supported for predict. Call ListModels to see the list of available models and their supported methods.

In [23]:

q = """Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Draw this cover in the syle of an 8-bit video game using imagen3.
"""
await process_and_display_RAG_comparison(model,q,the_attached_file)









 ------ STD RAG -------------------------






Based on the provided text, the most emotionally charged and important scene for the book cover, focusing on Alex Mercer's strongest relationships and key plot locations, would be the final scene in Chapter 15. This is where the team successfully establishes a back-and-forth communication with the alien intelligence.  It marks the culmination of their efforts, showcasing the team's unity and the momentous nature of their achievement.  The Dulce Base command center serves as a crucial location throughout the plot.

I can't actually *draw* the cover using Imagen-3, as I am a text-based AI.  However, I can provide you with a description suitable for prompting Imagen-3 to generate the 8-bit artwork:


**Imagen-3 Prompt:**

"8-bit video game cover art.  Title: Operation: Dulce.  Scene: Four characters, Alex Mercer (leader, determined expression), Jordan Hayes (scientist, focused expression), Taylor Cruz (military officer, serious expression), Sam Rivera (tech expert, excited expression) huddled around a bank of glowing 8-bit computer monitors in a dimly lit underground Dulce base command center.  Monitors display a mix of complex alien symbols and human data. Pixel art style, reminiscent of classic NES or SNES games.  Color palette: dark blues, greens, and accented with bright glowing colors from the monitors. Dramatic lighting.  Include a small spaceship icon in the upper right corner.  Text should be bold and easily legible."


This prompt should give Imagen-3 enough information to generate a compelling 8-bit style cover image capturing the essence of the climax of the story. Remember to experiment with different keywords and variations of this prompt to achieve your desired aesthetic.


------ GRAPH RAG -------------------------



Here's a description of a book cover scene, suitable for an 8-bit video game style, based on the provided text and knowledge graph:


**Scene Choice:** The most emotionally charged and important scene, considering Alex Mercer's strongest relationships (Jordan Hayes and Sam Rivera) and key locations (Dulce Base), is the moment in the Dulce Base command center when the team first successfully receives and interprets a clear message from the alien intelligence. This moment represents the culmination of their efforts, the beginning of interspecies communication, and a turning point in the narrative, full of hope and apprehension.


**Book Cover Description (8-bit style):**

The cover depicts a pixelated scene within the Dulce Base command center.  

* **Foreground:** Alex Mercer (sprite with determined expression, perhaps holding a data pad) stands slightly behind and to the side of Jordan Hayes, who is hunched over a brightly glowing terminal displaying pixelated alien symbols. Sam Rivera is visible in the background, partially obscured, working on their own console that displays cascading code.

* **Background:** The command center is depicted in a dark, 8-bit style, with simple pixelated representations of consoles and monitors showing various screens of data.  There's a hint of a metallic, underground texture to the walls.   The style should aim for a retro feel, reminiscent of classic 8-bit RPGs.

* **Color Palette:**  A limited palette of colors should be used: dark blues and greys for the background, bright blue, green, and perhaps a hint of orange for the consoles and monitors. Alex’s sprite could be in shades of khaki or grey.  The alien symbols could use a striking color, maybe purple or red, to stand out.


* **Title:**  "Operation: Dulce" in a bold, pixelated font that evokes 8-bit game titles.   The subtitle could be something like "First Contact" or "The Song of the Stars."


* **Overall Feel:**  The image should convey a sense of both tension and wonder, hinting at the momentous nature of the event depicted.  The pixel art style should be crisp and clear, emphasizing the retro aesthetic.


**Imagen prompt (Example):**

"8-bit pixel art book cover, retro RPG style, dark blue background, title 'Operation: Dulce' in bold pixel font, subtitle 'First Contact', scene: dimly lit command center, Alex Mercer (khaki/grey pixel sprite, determined) slightly behind Jordan Hayes (pixel sprite, focused on glowing blue terminal showing pixelated purple alien symbols), Sam Rivera (pixel sprite, working at console displaying cascading green code in background), metallic texture on walls."


This prompt can be further refined to get the desired level of detail and style using Imagen or similar image generation tools.  Remember to experiment with different keywords to achieve the specific visual representation you're aiming for.  The key is to evoke the retro 8-bit feel while maintaining clarity and storytelling in the image itself.
